1. Identify all NAs in the data.
2. Summary Statistics Table
3. Scatterplots
4. Heat Maps (Correlations)
5. Hypothesis Testing
6. Regression

In [46]:
import pandas as pd
import numpy as np
walking_data = pd.read_csv("data_sources/walkable-cities.csv",encoding='latin-1')
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,households,mean_household_income,mean_income,median_household_income,cost_of_living_index,poverty,unemployment_rate,median_aqi,violent_crime,property_crime
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076.0,157.084761,...,29136.0,79325.0,29770.0,48531.0,91.9,73331.0,NaN,NaN,NaN,NaN
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000.0,380.801025,...,87570.0,58012.0,27211.0,36614.0,90.7,189176.0,7.4,51.0,NaN,NaN
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104.0,140.785624,...,NaN,NaN,NaN,NaN,88.6,NaN,NaN,39.0,NaN,NaN
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184.0,232.650562,...,30301.0,70513.0,31168.0,45088.0,86.3,70074.0,5.3,NaN,NaN,NaN
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896.0,124.490626,...,36721.0,124562.0,50046.0,99276.0,NaN,91882.0,3.5,51.0,114.0,1922.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000.0,40.080067,...,30885.0,60267.0,24591.0,47861.0,90.9,75661.0,6.4,43.0,291.0,990.0
758,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250.0,66.102904,...,31139.0,88781.0,38381.0,69533.0,NaN,68468.0,NaN,43.0,67.0,690.0
759,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300.0,29.477371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,211.0,1686.0
760,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150.0,68.757037,...,NaN,NaN,NaN,NaN,94.2,NaN,NaN,42.0,169.0,1677.0


In [47]:
#1. Identify all NAs in the data.
col_na = walking_data.isna().sum().to_frame().reset_index(level=0).set_axis(["variable","sum_na"], axis="columns", copy=False)
row_na = walking_data["city_state"]
row_na = pd.concat([row_na, walking_data.isna( ).sum(axis="columns")], axis=1)
row_na = row_na.set_axis(["city_state","sum_na"],axis="columns", copy=False)

col_na.to_csv("column_na_orig.csv", index = False)
row_na.to_csv("row_na_orig.csv", index = False)

nrow = len(walking_data)
mcol = len(walking_data.columns)



